In [22]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt #for making figures
%matplotlib inline

In [23]:
words = open("names.txt", "r").read().splitlines()
#String to integer and integer to string mappings have been created
chars = sorted(list(set( "".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [24]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:])     # 10%


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [25]:
def cmp(s, dt, t):
    if t.grad is None:
        print(f"{s:15s} | Gradient is None")
        return
    if dt.shape != t.grad.shape:
        print(f"{s:15s} | Shape mismatch: {dt.shape} vs {t.grad.shape}")
        return
    ex = torch.equal(dt, t.grad)
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')


### Slightly chnaged initialisation compared to lecture 02
The reason we want to have all these small numbers instead of e.g 0 like previously is, that we want to ensure, that any incorrect implementation of the backpropagation is clearly visible

In [26]:
n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 64  # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)  # for reproducibility
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) *(5/3) / (n_embd * block_size)**0.5 #Kaiming initialisation
b1 = torch.randn(n_hidden, generator=g) * 0.1 #Not needed anymore, look in the training code
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

#Calcuklating the mean and std dev of the entire training set during training
bngain= torch.randn((1, n_hidden)) * 0.1 + 1.0 # Mean of normal distribution will be intially around 1
bnbias = torch.randn((1, n_hidden))*0.1 # std dev of normal distribution will be intially arounf 0 


parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))  # number of parameters in total
for p in parameters:
    p.requires_grad = True

4073


In [27]:
batch_size = 32
n = batch_size #using a shorter variable for convienience lmao
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

In [28]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()


#PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani, embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3596, grad_fn=<NegBackward0>)

# Manual Backpropagation

Starting from the end create a derivative in respect for every variable line by line. Make sure to apply chain rule eg: b = ax +h -> dL/da = dL/db * db/da.
If a variable is on the left side multiple times, our graph is "split" into 2 parts, we therefore need to sum the local derivatives eg:
b = ax + h
c = ay 

-> dL/da = dL/db * db/da + dL/dc * dc/da


### Broadcasting and Sum

Every time we have a broadcasting [1, 64] -> [32, 64] in the forward pass we will have a sumation in the backward pass ```[32, 64].sum(0, keepdim = True) ->[1,64]``` </br>
And vice versa:</br>
Sumation in the forward pass -> broadcasting in the backwardpass
[32, 64] -> [1, 64] and ```[1, 64] * torch.ones_like([32,64])```

### Exkurs: Bessels correction

<img src = "besselscorrection.png"/>
<img src = "besselcorr2.png"/>

Therefore we calculate the variance of the mini batch using the unbiased variance 
```python
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
```
The math proof is in the [paper](https://math.oxford.emory.edu/site/math117/besselCorrection/)

In [29]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
dprobs = (1.0 / probs) * dlogprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim = True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = (- counts_sum ** -2) *  dcounts_sum_inv
dcounts += dcounts_sum * torch.ones_like(counts)
dnorm_logits = dcounts * norm_logits.exp()
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim = True)
temp = torch.zeros([32,27])
temp[torch.arange(0,n),torch.reshape(logits.max(1, keepdim=True).indices, (1,32))] = 1
dlogits += temp * dlogit_maxes 
dh = dlogits @ torch.t(W2)
dW2 = torch.t(h) @dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - torch.tanh(hpreact) ** 2.0) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim = True)
dbnvar = (-0.5 * (bnvar + 1e-5)** -1.5) * dbnvar_inv
dbndiff2 = (1.0/(n-1.0)) * dbnvar * torch.ones_like(bndiff2)
dbndiff = 2 * bndiff * dbndiff2
dbndiff += bnvar_inv * dbnraw
dhprebn = dbndiff.clone()
dbnmeani = -dbndiff.clone().sum(0, keepdim = True)
dhprebn += 1.0/n * torch.ones_like(hprebn)*  dbnmeani
dembcat = dhprebn @ torch.t(W1)
dW1 = torch.t(embcat) @dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.clone().view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]



cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)


logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [30]:
from graphviz import Digraph

# Initialize the graph
dot = Digraph()
dot.attr(rankdir='LR')  # Set graph direction to left-to-right

# Add nodes for each operation
dot.node('Xb', 'Xb')
dot.node('C', 'C')
dot.node('emb', 'emb = C[Xb]')
dot.node('embcat', 'embcat = emb.view(...)')
dot.node('W1', 'W1')
dot.node('b1', 'b1')
dot.node('hprebn', 'hprebn = embcat @ W1 + b1')
dot.node('bnmeani', 'bnmeani = mean(hprebn)')
dot.node('bndiff', 'bndiff = hprebn - bnmeani')
dot.node('bndiff2', 'bndiff2 = bndiff^2')
dot.node('bnvar', 'bnvar = var(hprebn)')
dot.node('bnvar_inv', 'bnvar_inv = (bnvar + 1e-5)^-0.5')
dot.node('bnraw', 'bnraw = bndiff * bnvar_inv')
dot.node('bngain', 'bngain')
dot.node('bnbias', 'bnbias')
dot.node('hpreact', 'hpreact = bngain * bnraw + bnbias')
dot.node('tanh', 'h = tanh(hpreact)')
dot.node('W2', 'W2')
dot.node('b2', 'b2')
dot.node('logits', 'logits = h @ W2 + b2')
dot.node('logit_maxes', 'logit_maxes = logits.max(...)')
dot.node('norm_logits', 'norm_logits = logits - logit_maxes')
dot.node('counts', 'counts = exp(norm_logits)')
dot.node('counts_sum', 'counts_sum = sum(counts)')
dot.node('counts_sum_inv', 'counts_sum_inv = 1 / counts_sum')
dot.node('probs', 'probs = counts * counts_sum_inv')
dot.node('logprobs', 'logprobs = log(probs)')
dot.node('loss', 'loss = -logprobs.mean()')

# Add forward pass edges
dot.edge('Xb', 'emb', label='Embedding lookup')
dot.edge('C', 'emb', label='Embedding matrix')
dot.edge('emb', 'embcat', label='Flatten')
dot.edge('embcat', 'hprebn', label='Matrix multiply')
dot.edge('W1', 'hprebn', label='Weights')
dot.edge('b1', 'hprebn', label='Bias')
dot.edge('hprebn', 'bnmeani', label='Mean')
dot.edge('hprebn', 'bndiff', label='Difference')
dot.edge('bnmeani', 'bndiff', label='Subtraction')
dot.edge('bndiff', 'bndiff2', label='Square')
dot.edge('bndiff2', 'bnvar', label='Variance')
dot.edge('bnvar', 'bnvar_inv', label='Inverse sqrt')
dot.edge('bnvar_inv', 'bnraw', label='Scale')
dot.edge('bndiff', 'bnraw', label='Scale')
dot.edge('bnraw', 'hpreact', label='Shift and scale')
dot.edge('bngain', 'hpreact', label='Gain')
dot.edge('bnbias', 'hpreact', label='Bias')
dot.edge('hpreact', 'tanh', label='Tanh activation')
dot.edge('tanh', 'logits', label='Matrix multiply')
dot.edge('W2', 'logits', label='Weights')
dot.edge('b2', 'logits', label='Bias')
dot.edge('logits', 'logit_maxes', label='Max for stability')
dot.edge('logit_maxes', 'norm_logits', label='Subtract max')
dot.edge('logits', 'norm_logits', label='Subtract max')
dot.edge('norm_logits', 'counts', label='Exponentiate')
dot.edge('counts', 'counts_sum', label='Sum')
dot.edge('counts_sum', 'counts_sum_inv', label='Inverse')
dot.edge('counts', 'probs', label='Normalize')
dot.edge('counts_sum_inv', 'probs', label='Normalize')
dot.edge('probs', 'logprobs', label='Log')
dot.edge('logprobs', 'loss', label='Mean and negate')

# Add backward pass hints directly next to corresponding nodes
dot.node('grad_loss', 'dL/dloss', shape='ellipse', color='red')
dot.edge('loss', 'grad_loss', label='Backward pass')

dot.node('grad_logprobs', 'dL/dlogprobs', shape='ellipse', color='red')
dot.edge('grad_loss', 'grad_logprobs', label='Backward pass')

dot.node('grad_probs', 'dL/dprobs', shape='ellipse', color='red')
dot.edge('grad_logprobs', 'grad_probs', label='Backward pass')

dot.node('grad_counts_sum_inv', 'dL/dcounts_sum_inv', shape='ellipse', color='red')
dot.edge('grad_probs', 'grad_counts_sum_inv', label='Backward pass')

dot.node('grad_counts', 'dL/dcounts', shape='ellipse', color='red')
dot.edge('grad_probs', 'grad_counts', label='Backward pass')
dot.edge('grad_counts_sum_inv', 'grad_counts', label='Backward pass')

dot.node('grad_norm_logits', 'dL/dnorm_logits', shape='ellipse', color='red')
dot.edge('grad_counts', 'grad_norm_logits', label='Backward pass')

dot.node('grad_logits', 'dL/dlogits', shape='ellipse', color='red')
dot.edge('grad_norm_logits', 'grad_logits', label='Backward pass')

dot.node('grad_tanh', 'dL/dtanh', shape='ellipse', color='red')
dot.edge('grad_logits', 'grad_tanh', label='Backward pass')

dot.node('grad_hpreact', 'dL/dhpreact', shape='ellipse', color='red')
dot.edge('grad_tanh', 'grad_hpreact', label='Backward pass')

dot.node('grad_bnraw', 'dL/dbnraw', shape='ellipse', color='red')
dot.edge('grad_hpreact', 'grad_bnraw', label='Backward pass')

dot.node('grad_bndiff', 'dL/dbndiff', shape='ellipse', color='red')
dot.edge('grad_bnraw', 'grad_bndiff', label='Backward pass')

dot.node('grad_bnvar_inv', 'dL/dbnvar_inv', shape='ellipse', color='red')
dot.edge('grad_bnraw', 'grad_bnvar_inv', label='Backward pass')

dot.node('grad_bnvar', 'dL/dbnvar', shape='ellipse', color='red')
dot.edge('grad_bnvar_inv', 'grad_bnvar', label='Backward pass')

dot.node('grad_bndiff2', 'dL/dbndiff2', shape='ellipse', color='red')
dot.edge('grad_bnvar', 'grad_bndiff2', label='Backward pass')

dot.edge('grad_bndiff2', 'grad_bndiff', label='Backward pass')
dot.edge('grad_bndiff', 'grad_hprebn', label='Backward pass')
dot.edge('grad_hprebn', 'bnmeani', label='Backward pass')
dot.edge('grad_hprebn', 'hprebn', label='Backward pass')

dot.node('grad_embcat', 'dL/dembcat', shape='ellipse', color='red')
dot.edge('grad_hprebn', 'grad_embcat', label='Backward pass')

dot.node('grad_emb', 'dL/demb', shape='ellipse', color='red')
dot.edge('grad_embcat', 'grad_emb', label='Backward pass')

dot.node('grad_Xb', 'dL/dXb', shape='ellipse', color='red')
dot.edge('grad_emb', 'grad_Xb', label='Backward pass')

# Render the graph to a high-resolution PNG
dot.format = 'png'
dot.attr(dpi='300')
dot.render('backpropagation_graph', view=True)

print("Graph saved as backpropagation_graph.png")


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.77001 to fit


Graph saved as backpropagation_graph.png


In [31]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.359632968902588 diff: 2.384185791015625e-07


In [32]:
logits.shape

torch.Size([32, 27])

In [33]:
# backward pass
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 5.3551048040390015e-09
